# Analysis of HBL - Step by Step

In [1]:
import argparse
import math
import os

import warnings
warnings.filterwarnings('ignore')
# the code below simple hides some warnings we don't want to see
# import sklearn
# from sklearn.exceptions import DataConversionWarning
# warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import torch
# from torch.utils.tensorboard import SummaryWriter

from helper import pmath
from helper.helper import get_optimizer, load_dataset 
from helper.hyperbolicLoss import PeBusePenalty
from models.cifar import resnet as resnet_cifar


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from tqdm import tqdm

import pathlib

sns.set_style('darkgrid')


import numpy as np
import argparse
import math
import os
import torch
import torch.optim as optim
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import torch.optim as optim

import torch
from torchmetrics import Accuracy
from torchmetrics import ConfusionMatrix
from torchmetrics import F1Score

# HBL
from helper import pmath
from helper.helper import get_optimizer, load_dataset 
from helper.hyperbolicLoss import PeBusePenalty

In [3]:
model = resnet_cifar.ResNet(32, 10, 1, 1)


# Load data.
batch_size = 128
kwargs = {'num_workers': 32, 'pin_memory': True}
trainloader, testloader = load_dataset("cifar100", 'data/', batch_size, kwargs)


Files already downloaded and verified


In [4]:
for bidx, (data, target) in enumerate(trainloader):
    print(type(bidx))
    print(type(data))
    print(type(target))
    break

<class 'int'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [24]:
for bidx, (data, target) in enumerate(trainloader):
    print(bidx)
    print(data[0][0])
    print(target[0:3])
    print(target)
    break

[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati

0
tensor([[-1.8989, -1.8989, -1.8989,  ...,  1.8318,  1.8318,  1.8318],
        [-1.8989, -1.8989, -1.8989,  ...,  1.8318,  1.8318,  1.8318],
        [-1.8989, -1.8989, -1.8989,  ...,  1.8318,  1.8318,  1.8318],
        ...,
        [-1.8989, -1.8989, -1.8989,  ..., -1.8989, -1.8989, -1.8989],
        [-1.8989, -1.8989, -1.8989,  ..., -1.8989, -1.8989, -1.8989],
        [-1.8989, -1.8989, -1.8989,  ..., -1.8989, -1.8989, -1.8989]])
tensor([ 4, 17, 35])
tensor([ 4, 17, 35, 70, 84, 30, 30, 17, 84,  7, 88, 25, 32, 79, 55, 55, 20, 80,
        53, 69,  4, 35, 36, 91, 25, 68, 70, 22, 83, 46, 28,  5, 91, 51, 36, 26,
        37, 37, 65, 83,  4, 84, 43, 30, 41, 47, 18, 82, 79, 34, 41, 91, 13, 34,
        67,  9, 89, 63, 51,  7, 35, 36, 34, 93, 35, 87, 17, 22, 51, 56, 57, 97,
        22, 74,  8, 62, 86, 53, 23, 54, 36, 14, 55, 73, 66, 93, 66, 91, 97, 48,
        40, 45, 23, 15, 16, 52, 55, 56, 43, 54,  4, 35, 32, 70, 80, 10, 45, 71,
        67, 64, 76, 96, 40, 93, 44, 96, 81, 98, 67,  3,  6, 61,

In [5]:
hpnfile = 'prototypes/prototypes-10d-10c.npy'
out_dims = int(hpnfile.split("/")[-1].split("-")[1][:-1]) # if -10d-1-c --> 10 from the d
planes = out_dims

In [6]:
dims = 300 

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(dims, planes) # Input Layer
        self.fc2 = nn.Linear(planes, planes) # Hidden Layer 1
        self.fc3 = nn.Linear(planes, planes) # Hidden Layer 2
        self.fc4 = nn.Linear(planes, out_dims) # Output Layer
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x # F.log_softmax(x, dim=1)  We want to sum the classes - across columns

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=300, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [7]:
target_tmp = target # .cuda()
target_tmp

tensor([48, 92,  5,  8, 82, 74, 52, 78, 83, 80, 64, 26, 14, 95, 38, 39, 58, 10,
        78, 48, 24, 54, 91, 68, 36, 30, 37, 28, 78, 11, 85, 40, 62,  7, 13,  3,
        21, 72, 17, 39, 88, 29, 80, 99, 80, 96, 70, 63,  1, 14, 77, 10,  2, 68,
        29, 37, 57, 42, 93, 83, 90, 27, 98, 37, 39, 43, 97, 59, 42, 59, 50, 74,
        87, 89, 15, 99, 19, 13, 26, 57, 45, 81,  1, 43, 20, 20, 82, 11, 67, 57,
        46, 77, 14, 63, 44, 71, 22, 18, 81, 35, 21, 65, 90, 19, 81,  2, 56, 38,
        76, 89, 29, 50, 56, 50, 32, 40,  8, 52, 10, 12, 82, 83, 51, 18, 10, 87,
        74, 11])

In [8]:
choose_penalty = 'dim'
mult = 0.001
loss_function = PeBusePenalty(out_dims, penalty_option= choose_penalty, mult=mult) # .cuda()
optimizer = optim.Adam(net.parameters(), lr=1e-3)
model = resnet_cifar.ResNet(32, out_dims, 1, 1)

In [18]:
# target = model.polars[target]
data = torch.autograd.Variable(data) # .cuda()
# target = torch.autograd.Variable(target) # .cuda()
# Compute outputs and losses.
output = model(data)
output_exp_map = pmath.expmap0(output, c=1)

loss_function = loss_function(output_exp_map, target)

In [31]:
pmath.expmap0(torch.tensor(1.), c = 0.001)

tensor(0.9997)

In [20]:
output

tensor([[-0.7535, -0.2151, -0.4674,  ..., -0.7364,  0.9627, -0.0478],
        [-0.7469, -0.0850, -0.6503,  ..., -0.6085,  1.0488, -0.0679],
        [-0.5652, -0.0087, -0.5199,  ..., -0.4376,  0.8285,  0.0547],
        ...,
        [-1.1835, -0.1630, -0.1258,  ..., -0.5741,  0.7041, -0.5325],
        [-0.5684,  0.2343, -0.4232,  ..., -0.5723,  0.9004,  0.0164],
        [-0.5657,  0.1740, -0.4012,  ..., -0.4063,  0.9182,  0.0851]],
       grad_fn=<AddmmBackward0>)

In [21]:
output_exp_map

tensor([[-0.2804, -0.0800, -0.1739,  ..., -0.2740,  0.3582, -0.0178],
        [-0.3038, -0.0346, -0.2645,  ..., -0.2475,  0.4266, -0.0276],
        [-0.2535, -0.0039, -0.2331,  ..., -0.1962,  0.3715,  0.0245],
        ...,
        [-0.3428, -0.0472, -0.0364,  ..., -0.1663,  0.2039, -0.1542],
        [-0.2660,  0.1096, -0.1980,  ..., -0.2678,  0.4213,  0.0077],
        [-0.2544,  0.0782, -0.1804,  ..., -0.1827,  0.4129,  0.0382]],
       grad_fn=<DivBackward0>)

In [22]:
target

1

In [15]:
model.polars

1

In [ ]:
model.train()
avgloss, avglosscount, newloss, acc, newacc = 0., 0., 0., 0., 0.

In [49]:
# Set mode to training.
model.train()
avgloss, avglosscount, newloss, acc, newacc = 0., 0., 0., 0., 0.

for bidx, (data, target) in enumerate(trainloader):
    target_tmp = target #.cuda()
    print(target_tmp)

    target = model.polars[target]
    # data = torch.autograd.Variable(data) # .cuda()
    # target = torch.autograd.Variable(target) # .cuda()
    # # Compute outputs and losses.
    # output = model(data)
    # output_exp_map = pmath.expmap0(output, c=1)

    # loss_function = loss_function(output_exp_map, target)
    break

[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati

tensor([21,  8, 59, 41, 46, 89, 51, 88, 16, 84, 30, 71, 64, 64, 95, 46, 18, 82,
        60, 23, 18, 60,  0, 23, 66, 28, 95, 75, 19, 50, 28, 35, 83, 80, 96, 50,
        78,  1, 55, 52, 78, 42, 51, 36, 48, 59, 89, 45, 95, 69, 83, 66, 45, 16,
        15, 83, 24, 94, 86, 60, 98,  2, 92, 56, 64, 56,  3, 79, 82,  0,  7, 17,
        38, 87, 15,  0, 54,  8, 76, 99, 94, 92, 49, 51, 20,  4, 22, 74, 27, 79,
        58, 95, 61, 51, 45, 65, 68, 72, 39, 85, 48, 74, 59, 37,  3, 19, 87, 47,
         6, 95, 34, 67, 56, 93, 80, 83, 66, 67, 10, 82, 12, 76,  7,  5, 68, 57,
        14, 86])


TypeError: 'int' object is not subscriptable